In [7]:
%load_ext autoreload
%autoreload 2

import pandas as pd

from forgeflow import paths, io, clean

print("Project root:", paths.get_project_root())
print("Raw data dir:", paths.get_data_dir('raw'))




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Project root: /workspaces/forgeflow-manufacturing-analytics
Raw data dir: /workspaces/forgeflow-manufacturing-analytics/data/raw


In [19]:
from pathlib import Path

raw_dir = paths.RAW_DIR
print("RAW_DIR:", raw_dir)

for p in sorted(raw_dir.rglob("*.csv")):
    print(p.relative_to(raw_dir))


RAW_DIR: /workspaces/forgeflow-manufacturing-analytics/data/raw
olist/olist_customers_dataset.csv
olist/olist_geolocation_dataset.csv
olist/olist_order_items_dataset.csv
olist/olist_order_payments_dataset.csv
olist/olist_order_reviews_dataset.csv
olist/olist_orders_dataset.csv
olist/olist_products_dataset.csv
olist/olist_sellers_dataset.csv
olist/product_category_name_translation.csv
secom/uci-secom.csv


In [20]:
orders = io.read_raw_csv('olist/olist_orders_dataset.csv')
secom = io.read_raw_csv('secom/uci-secom.csv')

print("orders shape:", orders.shape)
print("secom shape:", secom.shape)

orders shape: (99441, 8)
secom shape: (1567, 592)


In [21]:
secom.head()

,Time,0,1,2,3,4,5,6,7,8,...,581,582,583,584,585,586,587,588,589,Pass/Fail
0,2008-07-19 11:55:00,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,...,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN,-1
1,2008-07-19 12:32:00,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,...,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045,-1
2,2008-07-19 13:17:00,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,...,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602,1
3,2008-07-19 14:43:00,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.4882,...,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432,-1
4,2008-07-19 15:22:00,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5031,...,NaN,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432,-1


In [22]:
orders_clean = clean.standardize_columns(orders)
secom_clean = clean.standardize_columns(secom)

orders_clean.columns[:20], secom_clean.columns[:20]


(Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
        'order_approved_at', 'order_delivered_carrier_date',
        'order_delivered_customer_date', 'order_estimated_delivery_date'],
       dtype='object'),
 Index(['time', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
        '12', '13', '14', '15', '16', '17', '18'],
       dtype='object'))

In [24]:
orders = io.read_raw_csv("olist/olist_orders_dataset.csv")
customers = io.read_raw_csv("olist/olist_customers_dataset.csv")
items = io.read_raw_csv("olist/olist_order_items_dataset.csv")

# Standardize column names
orders = clean.standardize_columns(orders)
customers = clean.standardize_columns(customers)
items = clean.standardize_columns(items)

orders.head(), customers.head(), items.head()

(                           order_id                       customer_id  \
 0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
 1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
 2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
 3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
 4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   
 
   order_status order_purchase_timestamp    order_approved_at  \
 0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
 1    delivered      2018-07-24 20:41:37  2018-07-26 03:24:27   
 2    delivered      2018-08-08 08:38:49  2018-08-08 08:55:23   
 3    delivered      2017-11-18 19:28:06  2017-11-18 19:45:59   
 4    delivered      2018-02-13 21:18:39  2018-02-13 22:20:29   
 
   order_delivered_carrier_date order_delivered_customer_date  \
 0          2017-10-04 19:55:00           2017-10-10 21:25:13   
 1          2018-07-26 14:31:00 

In [25]:
orders.info()
orders.describe(include="all").T.head(20)

customers.info()
items.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null 